# Chapter 2.6. Intro to TensorFlow and Keras

Original code in Nikolenko book was made in **TensorFlow 1.x (TF1)**, but here I will convert examples to **TensorFlow 2.0 (TF2)**

In [1]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf

## 1. Simple Regression using TensorFlow 2

$$y = k_{true} * x + b_{true} + \epsilon$$

$$\epsilon - normal noise (N(0, 2))$$

Let's try to recover coefs `k_true`, `b_true`

In [2]:
# set constants
n_samples = 1000
batch_size = 100
num_steps = 2000
display_step = 100

k_true = 2
b_true = 1

# generate data
X_data = np.random.uniform(1,10,(n_samples, 1))
y_data = k_true * X_data + b_true + np.random.normal(0, 2, (n_samples, 1))

# initialize coefs
k = tf.Variable(tf.random.normal((1,1), dtype='float64'), name='slope')
b = tf.Variable(tf.zeros((1,), dtype='float64'), name='bias')


# create model class
class MyModel():
    
    def __init__(self, k, b):
        self.k = k
        self.b = b
        
    def __call__(self, X):
        return tf.matmul(X, self.k) + self.b
    
# define loss
def mse_loss(y_true, y_pred):
    return tf.reduce_sum(tf.square(y_true - y_pred))

# define forward step
def train(model, X, y, optimizer):
    """
    One step of optimization
    
    params:
        model: model object ot train
        X: data
        y: target 
    return:
        loss: loss value
    """
    with tf.GradientTape() as t:
        y_pred = model(X)
        loss = mse_loss(y, y_pred)
        
    grads = t.gradient(loss, [model.k, model.b])
    optimizer.apply_gradients(zip(grads,[model.k, model.b]))
    
    return loss

In [3]:
optimizer = tf.optimizers.SGD(learning_rate=1e-4)
model = MyModel(k, b)

# running optimization
for i in range(num_steps):
    
    # select data batch
    indices = np.random.choice(n_samples, batch_size)
    X_batch, y_batch = X_data[indices], y_data[indices]
    
    loss_val = train(model, X_batch, y_batch, optimizer)
    
    # output info
    if (i+1) % display_step == 0:
        print(f'Epoch {i+1}: loss={loss_val}, k={model.k.numpy()}, b={model.b.numpy()}')

Epoch 100: loss=405.2075125448249, k=[[2.03974921]], b=[0.48782402]
Epoch 200: loss=432.12792702674767, k=[[2.03229947]], b=[0.67374956]
Epoch 300: loss=423.07094733658084, k=[[2.00657784]], b=[0.79454969]
Epoch 400: loss=403.640519587412, k=[[2.04567009]], b=[0.88216861]
Epoch 500: loss=379.5927577813817, k=[[2.00102378]], b=[0.91664975]
Epoch 600: loss=499.99162499952297, k=[[1.99153073]], b=[0.95919205]
Epoch 700: loss=367.5354460205415, k=[[2.00231788]], b=[0.99361275]
Epoch 800: loss=367.7656598664735, k=[[1.98794238]], b=[1.02533584]
Epoch 900: loss=360.83962584540876, k=[[2.01658998]], b=[1.00642175]
Epoch 1000: loss=339.1165403425979, k=[[1.95835814]], b=[1.00355341]
Epoch 1100: loss=320.84698288482485, k=[[1.97359641]], b=[0.99783332]
Epoch 1200: loss=513.1217162002757, k=[[1.97291515]], b=[1.00958784]
Epoch 1300: loss=382.31173302154235, k=[[1.97768824]], b=[0.99861147]
Epoch 1400: loss=408.0682098660934, k=[[2.0017979]], b=[1.04160361]
Epoch 1500: loss=407.6285016343411, k=[

We got approximate values `k = 1.94` and `b = 0.96`.


## 2. Logistic Regression using TF2 Keras api

- Generate some data;
- Define simple logistic regression (1 dense layer with sigmoid activation);
- Train model

In [5]:
def sampler(n, x, y):
    return np.random.normal(size=[n,2]) + [x, y]

def sample_data(n=1000, p0=(-1., -1.), p1=(1., 1.)):
    zeros, ones = np.zeros((n, 1)), np.ones((n, 1))
    labels = np.vstack([zeros, ones])
    
    z_sample = sampler(n, x=p0[0], y=p0[1])
    o_sample = sampler(n, x=p1[0], y=p1[1])
    
    return np.vstack([z_sample, o_sample]), labels

X_train, Y_train = sample_data()
X_test, Y_test = sample_data(100)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(2000, 2) (2000, 1)
(200, 2) (200, 1)


In [14]:
# define model
model = tf.keras.Sequential(
    tf.keras.layers.Dense(1, activation='sigmoid')
)
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [12]:
# train model
model.fit(X_train, Y_train, epochs=50, batch_size=128, validation_data=(X_test, Y_test))

Train on 2000 samples, validate on 200 samples
Epoch 1/50
2000/2000 [==============================] - 0s 12us/sample - loss: 0.1973 - accuracy: 0.9225 - val_loss: 0.1826 - val_accuracy: 0.9400
Epoch 2/50
2000/2000 [==============================] - 0s 9us/sample - loss: 0.1973 - accuracy: 0.9225 - val_loss: 0.1826 - val_accuracy: 0.9400
Epoch 3/50
2000/2000 [==============================] - 0s 9us/sample - loss: 0.1973 - accuracy: 0.9225 - val_loss: 0.1825 - val_accuracy: 0.9400
Epoch 4/50
2000/2000 [==============================] - 0s 9us/sample - loss: 0.1972 - accuracy: 0.9225 - val_loss: 0.1825 - val_accuracy: 0.9400
Epoch 5/50
2000/2000 [==============================] - 0s 10us/sample - loss: 0.1972 - accuracy: 0.9225 - val_loss: 0.1824 - val_accuracy: 0.9400
Epoch 6/50
2000/2000 [==============================] - 0s 9us/sample - loss: 0.1972 - accuracy: 0.9225 - val_loss: 0.1824 - val_accuracy: 0.9400
Epoch 7/50
2000/2000 [==============================] - 0s 9us/sample - los